In [1]:
#Importaremos los módulos necesarios
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from os import getenv

matplotlib.rcParams['figure.figsize'] = (15.0, 7.5)
from IPython.display import Audio
np.set_printoptions(precision=3)

import UPVlog
student_id=getenv("USER")
notebook_filename="ExamenEnero2025-P1" + student_id
mylogger=UPVlog.UPVlog(notebook_filename)


print("Estudiante: ", student_id)

Estudiante:  profesor


# Objetivo

El objetivo de este cuaderno es analizar varios sistemas sencillos vistos en clase mediante el desarrollo de funciones que implementen cada una de las partes.

* Retardo
* Propagación simple
* Sistema Eco

## Retardo 

Implementaremos una función que recibirá como argumentos:
* x: Muestras de una señal de audio (mono o stereo)
* D: Cantidad de muestras a retardar la señal

El resultado será **una señal de la misma duración** que la señal de entrada pero retardada.

Si por ejemplo, ====>    $x=[1,3,4,3,2,1,5,6,7]$

la salida deberá ser si el retardo es D=2: ===>   $y=[0,0,1,3,4,3,2,1,5]$

Observe que si $D==0$ la salida debe ser idéntica a la entrada

In [2]:
def retardo(x,D):
    '''
    x: array de numpy con muestras de entrada. Señal mono o stereo
    D: tamaño del promediador deslizante Debe ser entero
    Si 
       x=[1,3,4,3,2,1,5,6,7]
    para D=2
       y=[0,0,1,3,4,3,2,1,5]

    '''
    y=None

    assert isinstance(D,int)

    ### BEGIN SOLUTION
    if D==0:
        return x
    if x.ndim ==1:
        cabecera=np.zeros(D)
        resto=x[:-D]
    else:
        cabecera=np.zeros((D,x.shape[1]))
        resto=x[:-D,:]
    y=np.concatenate((cabecera,resto),axis=0)
    
    ### END SOLUTION
    return y


In [3]:
# Comprobacion por el estudiante: caso mono  Hay tests ocultos
x=np.random.randn(8)
### BEGIN TESTS
mylogger.log("Testing retardo")
y=retardo(x,3)
assert y is not None, "Función no implementada"
assert len(x)==len(y), "Tamaño de la salida incorrecto"
assert x[3]==y[6], "Retardo incorrecto"

# Comprobacion si D==0
z= retardo(x,0)
assert np.max(np.abs(z-x))<1e-5, "Si el retardo vale 0 no funciona bien"
### END TESTS

print("x=",x)
print("y=",y)

### BEGIN HIDDEN TESTS
def m_retardo(x,D):
    '''
    x: array de numpy con muestras de entrada. Señal mono o stereo
    D: tamaño del promediador deslizante
    Si 
       x=[1,3,4,3,2,1,5,6,7]
    para D=2
       y=[0,0,1,3,4,3,2,1,5]
    '''
    y=None
    if D==0:
        return y
    if x.ndim ==1:
        cabecera=np.zeros(D)
        resto=x[:-D]
    else:
        cabecera=np.zeros((D,x.shape[1]))
        resto=x[:-D,:]
    y=np.concatenate((cabecera,resto),axis=0)
    return y

xx=np.random.randn(200)
yy=retardo(xx,22)
zz=m_retardo(xx,22)
assert yy.shape == zz.shape
assert np.max(np.abs(yy-zz))< 1e-5
### END HIDDEN TESTS

mylogger.log("Success: Testing retardo")

x= [ 0.903  1.228  1.793 -1.26  -0.05  -1.4    0.923  0.282]
y= [ 0.     0.     0.     0.903  1.228  1.793 -1.26  -0.05 ]


In [4]:
# Comprobacion por el estudiante: caso stereo Hay tests ocultos
xstereo=np.random.randn(6,2)
ystereo=retardo(xstereo,2)
mylogger.log("Testing retardo stereo")
### BEGIN TESTS
assert ystereo is not None, "Función no implementada"
assert xstereo.shape==ystereo.shape, "Tamaño de la salida incorrecto"
assert np.max(np.abs(xstereo[1,:]-ystereo[3,:]))< 1e-5, "Retardo incorrecto"
### END TESTS

print("xstereo=",xstereo)
print("ystereo=",ystereo)

### BEGIN HIDDEN TESTS

xx=np.random.randn(300,2)
yy=retardo(xx,19)
zz=m_retardo(xx,19)
assert yy.shape == zz.shape
assert np.max(np.abs(yy-zz))< 1e-5
### END HIDDEN TESTS

mylogger.log("Sucess: Testing retardo stereo")

xstereo= [[ 0.263 -0.607]
 [ 0.097  0.311]
 [ 1.273  0.968]
 [-1.509 -0.366]
 [-0.757 -0.688]
 [-0.131 -2.256]]
ystereo= [[ 0.     0.   ]
 [ 0.     0.   ]
 [ 0.263 -0.607]
 [ 0.097  0.311]
 [ 1.273  0.968]
 [-1.509 -0.366]]


_____________________________________________-

## Propagación simple

El sistema que denominamos *propagación simple* no es más que la conexión en cascada de :

* Un retardo

* Un atenuador

Implementaremos:
* Una función "atenuador" que recibirá como parámetros la señal a atenuar (stereo o mono) y una atenuación **en dB** . La función internamente realizará una conversión de atenuación en dB a ganancia en amplitud
* Una función llamada "propagación simple"

In [5]:
def atenuador(x,atenuacion_dB):
    '''
    x: array de numpy con muestras de la señal a atenuar (mono o stereo)
    atenuacion_dB
    '''

    y=None
    ### BEGIN SOLUTION
    G= 10 ** (-atenuacion_dB/20)
    y= G * x
    ### END SOLUTION

    return y

In [6]:
# Comprobacion atenuador 
mylogger.log("Testing atenuador")
x=np.random.randn(100,2)
y=atenuador(x,40)
### BEGIN TESTS
assert y is not None, "funcion no implementada"
assert x.shape == y.shape, "Deben tener el mismo tamaño"

assert (np.mean(x**2)/np.mean(y**2) -10000) < 1e-3, "No ha calculado bien la ganancia"
### END TESTS

### BEGIN HIDDEN TESTS
x=np.random.randn(1000,2)
at=33
y=atenuador(x,at)
assert y is not None, "funcion no implementada"
assert x.shape == y.shape, "Deben tener el mismo tamaño"

assert np.max(np.abs(y-x/10**(at/20))) < 1e-3, "No ha calculado bien la ganancia"
def m_atenuador(x,atenuacion_dB):
    '''
    x: array de numpy con muestras de la señal a atenuar (mono o stereo)
    atenuacion_dB
    '''

    y=None

    G= 10 ** (-atenuacion_dB/20)
    y= G * x


    return y
### END HIDDEN TESTS

mylogger.log("Sucess: Testing atenuador")


In [7]:
def propagacion_simple(x,D,atenuacion_dB):
    '''
    x: array de numpy con muestras de la señal a atenuar (mono o stereo)
    D: retardo de propagacion en muestras. Debe ser un entero
    atenuacion_dB
    '''

    assert isinstance(D,int), "D debe ser entero"
    y= None

    ## Se recomienda usar las funciones anteriores 
    
    ### BEGIN SOLUTION
    z=retardo(x,D)
    y=atenuador(z,atenuacion_dB)
    ### END SOLUTION

    return y

In [8]:
# Comprobacion propagacion simple . 
# 
mylogger.log("Testing propagacion simple")
x=np.random.randn(8)
y=propagacion_simple(x,2,20)
print(" >>> Revise los valores que se imprimen para comprobar que hacen lo que se espera de un sistema que retarda dos muestras y atenua 20 dB")
print('x=',x)
print('y=',y)
print(">>>>>>>>>>>>>>>>>>>>>>>>" )    
### BEGIN TESTS
assert y is not None, "funcion no implementada"
assert x.shape == y.shape, "Deben tener el mismo tamaño"

### END TESTS

### BEGIN HIDDEN TESTS
x=np.random.randn(800,2)
ret=21
y=propagacion_simple(x,ret,19)

assert y is not None, "funcion no implementada"
assert x.shape == y.shape, "Deben tener el mismo tamaño"

assert np.max(np.abs( x[:-ret,:]/10**(19/20) - y[ret:,:])) < 1e-5
def m_propagacion_simple(x,D,atenuacion_dB):
    '''
    x: array de numpy con muestras de la señal a atenuar (mono o stereo)
    D: retardo de propagacion en muestras. Debe ser un entero
    atenuacion_dB
    '''

    assert isinstance(D,int), "D debe ser entero"
    y= None

    z=m_retardo(x,D)
    y=m_atenuador(z,atenuacion_dB)

    return y
### END HIDDEN TESTS
mylogger.log("Sucess: Testing propagacion simple")


 >>> Revise los valores que se imprimen para comprobar que hacen lo que se espera de un sistema que retarda dos muestras y atenua 20 dB
x= [-0.294  0.226  1.049 -1.333  0.646  0.773 -0.906 -1.43 ]
y= [ 0.     0.    -0.029  0.023  0.105 -0.133  0.065  0.077]
>>>>>>>>>>>>>>>>>>>>>>>>


__________________________

## Sistema Eco

Para finalizar el ejercicio, implementará, básandose en las funciones anteriores una función que implemente un sistema ECO, cuya salida es la suma:
* De la señal de entrada
* El resultado de una propagación simple

In [9]:
def simula_eco(x,D,atenuacion_dB):
    '''
    Dada una señal x devuelve a la salida el resultado de atravesar un sistema eco con los parámetros dados
    D: retardo debe ser entero positivo
    x: array de numpy stereo o mono
    '''

    assert isinstance(D,int), "El retardo debe ser entero"
    assert D >0, "El retardo debe ser positivo"

    y = None

    # Se sugiere usar las funciones anteriores
    ### BEGIN SOLUTION
    return x + propagacion_simple(x,D,atenuacion_dB)
    ### END SOLUTION
    return y

In [10]:
x=np.zeros(10)
x2=np.zeros((10,2))
x[0]=1
x2[0,0]=1
x2[1,1]=1
y=simula_eco(x,4,6)
y2=simula_eco(x2,3,6)
print(" >>> Revise los valores que se imprimen para comprobar que hacen lo que se espera de un sistema que retarda dos muestras y atenua 20 dB")
print('x=',x)
print('y=',y)
print('x2=',x2)
print('y2=',y2)
print(">>>>>>>>>>>>>>>>>>>>>>>>" )    
mylogger.log("Testing Eco")
### BEGIN TESTS
assert y is not None
assert x.shape == y.shape
assert y2 is not None
assert x2.shape is not None
### END TESTS

### BEGIN HIDDEN TESTS
def m_simula_eco(x,D,atenuacion_dB):
    '''
    Dada una señal x devuelve a la salida el resultado de atravesar un sistema eco con los parámetros dados
    retardo debe ser entero positivo
    x: array de numpy stereo o mono
    '''

    assert isinstance(D,int), "El retardo debe ser entero"
    assert D >0, "El retardo debe ser positivo"

    y = None

    # Se sugiere usar las funciones anteriores

    return x + m_propagacion_simple(x,D,atenuacion_dB)

    return y


x3=np.random.randn(1000,2)
D=int(102)
y3=simula_eco(x3,D,13)
z3=m_simula_eco(x3,D,13)
assert y3 is not None
assert y3.shape == x3.shape
assert y3.shape == z3.shape
assert np.max(np.abs(y3-z3))< 1e-6
### END HIDDEN TESTS
mylogger.log("Sucess: Testing Eco")

 >>> Revise los valores que se imprimen para comprobar que hacen lo que se espera de un sistema que retarda dos muestras y atenua 20 dB
x= [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y= [1.    0.    0.    0.    0.501 0.    0.    0.    0.    0.   ]
x2= [[1. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
y2= [[1.    0.   ]
 [0.    1.   ]
 [0.    0.   ]
 [0.501 0.   ]
 [0.    0.501]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]]
>>>>>>>>>>>>>>>>>>>>>>>>



_______________________________________________

La celda anterior debe devolver lo siguiente:

![eco](images/eco.png) 

______________________________________________